In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
medicines = pd.read_csv('/kaggle/input/medicine/medicine.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/medicine/medicine.csv'

In [ ]:
medicines.head()

In [ ]:
medicines.shape

In [ ]:
medicines.isnull().sum()

In [ ]:
medicines.dropna(inplace=True)

In [ ]:
medicines.duplicated().sum()

In [ ]:
medicines['Description']

In [ ]:
medicines['Description'].apply(lambda x:x.split())

In [ ]:
medicines['Reason'] = medicines['Reason'].apply(lambda x:x.split())
medicines['Description'] = medicines['Description'].apply(lambda x:x.split())

In [ ]:
medicines['Description'] = medicines['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
medicines['Description'] = medicines['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
medicines['tags'] = medicines['Description'] + medicines['Reason'] 

In [ ]:
new_df = medicines[['index','Drug_Name','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df

In [ ]:
!pip install nltk

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
!pip install -U scikit-learn scipy matplotlib

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',max_features=5000)

In [ ]:
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)  

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
cv.fit_transform(new_df['tags']).toarray().shape

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[1]

In [ ]:
def recommend(medicine):
    medicine_index = new_df[new_df['Drug_Name'] == medicine].index[0]
    distances = similarity[medicine_index]
    medicines_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in medicines_list:
        print(new_df.iloc[i[0]].Drug_Name)
    

In [ ]:
recommend("Paracetamol 125mg Syrup 60mlParacetamol 500mg Tablet 10'S")

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('medicine_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))